In [16]:
%%bash
mkdir ML_training_01
chown -R jupyter:jupyter /home/jupyter/ML_training_01

In [15]:
import os
os.chdir('ML_training_01/')

In [21]:
%%bash
# Iniciar un repositorio
git init

Initialized empty Git repository in /home/jupyter/ML_training_01/.git/


In [26]:
%%bash 
# revisar que commits tenemos para cargar
git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	Course_1.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [27]:

git add .

SyntaxError: invalid syntax (3827820173.py, line 1)